In [31]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import loguniform
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
df = pd.read_csv('data.csv', parse_dates=[1]).sort_values(by = "date")
df['date'] = df['date'].dt.to_period('M')
df = df[277:].reset_index(drop=True)

In [33]:
# For testing
df = df[df['date'] < '2008-01']

In [34]:
df.head()

,optid,date,secid,cp_flag,strike,bid,ask,volume,openint,impvol,...,vix,dhedged_return_mid,dhedged_return_spot,dhedged_return_spot_gamma,dhedged_return_mid_delev,dhedged_return_spot_delev,dhedged_return_spot_gamma_delev,IV_mness_deriv_1,IV_ttm_deriv_1,short_rate
0,10758593,1996-03,108105,C,630.0,20.000,20.500,7.0,2885.0,0.163628,...,16.54,-0.162051,-0.005116,-0.453583,-0.007654,-0.000242,-0.021423,-0.034178,0.000045,0.054624
1,10093854,1996-03,108105,P,650.0,17.500,18.250,126.0,2383.0,0.143119,...,16.54,-0.161114,-0.004490,-0.430225,-0.008292,-0.000231,-0.022141,-0.025132,0.000005,0.054624
2,10453184,1996-03,108105,P,675.0,33.625,34.625,0.0,202.0,0.133780,...,16.54,-0.032381,-0.001723,-0.212260,-0.002190,-0.000117,-0.014357,-0.008622,-0.000010,0.054624
3,11075738,1996-03,108105,P,650.0,35.000,36.500,13.0,2376.0,0.153038,...,16.54,-0.034062,-0.001898,-0.684611,-0.005364,-0.000299,-0.107810,-0.029598,0.000015,0.054624
4,11727207,1996-03,108105,C,655.0,30.250,31.250,7120.0,8000.0,0.146335,...,16.54,-0.019832,-0.000951,-0.160711,-0.001621,-0.000078,-0.013133,NaN,NaN,0.054624


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52914 entries, 0 to 52913
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype    
---  ------                           --------------  -----    
 0   optid                            52914 non-null  int64    
 1   date                             52914 non-null  period[M]
 2   secid                            52914 non-null  int64    
 3   cp_flag                          52914 non-null  object   
 4   strike                           52914 non-null  float64  
 5   bid                              52914 non-null  float64  
 6   ask                              52914 non-null  float64  
 7   volume                           52914 non-null  float64  
 8   openint                          52914 non-null  float64  
 9   impvol                           52914 non-null  float64  
 10  delta                            52914 non-null  float64  
 11  gamma                            52914 non-null  float

In [36]:
def get_summary(df_col):
    print(f'\nSummary of {df_col.name}:')
    print(f'Mean: {np.round( df_col.mean(),2)}')
    print(f'Median: {np.round( df_col.median(),2)}')
    print(f'Standard deviation: {np.round( df_col.std(),2)}')
    print(f'Number of observations: {np.round( df_col.count(),2)}')    

In [37]:
def add_months(date ,period_to_add=1):
    return (datetime.strptime(date, '%Y-%m') + relativedelta(months=period_to_add)).strftime('%Y-%m')

In [38]:
features = [ 'midprice', 'mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta']

In [39]:
print('\nSummary statistics for Call options\n')
df[df["cp_flag"] == 'C'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Call options


Summary of midprice:
Mean: 77.56
Median: 40.9
Standard deviation: 100.56
Number of observations: 22546

Summary of mness:
Mean: 0.41
Median: 0.25
Standard deviation: 1.16
Number of observations: 22546

Summary of ttm:
Mean: 203.15
Median: 126.0
Standard deviation: 192.24
Number of observations: 22546

Summary of embed_lev:
Mean: 18.22
Median: 12.47
Standard deviation: 16.89
Number of observations: 22546

Summary of impvol:
Mean: 0.21
Median: 0.19
Standard deviation: 0.11
Number of observations: 22546

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 22546

Summary of vega:
Mean: 211.09
Median: 166.22
Standard deviation: 173.31
Number of observations: 22546

Summary of theta:
Mean: -67.15
Median: -56.08
Standard deviation: 51.92
Number of observations: 22546


In [40]:
print('\nSummary statistics for Put options\n')

df[df["cp_flag"] == 'P'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Put options


Summary of midprice:
Mean: 73.9
Median: 27.75
Standard deviation: 116.3
Number of observations: 30368

Summary of mness:
Mean: -0.37
Median: -0.39
Standard deviation: 1.28
Number of observations: 30368

Summary of ttm:
Mean: 214.31
Median: 147.0
Standard deviation: 200.25
Number of observations: 30368

Summary of embed_lev:
Mean: 13.51
Median: 10.17
Standard deviation: 10.4
Number of observations: 30368

Summary of impvol:
Mean: 0.24
Median: 0.22
Standard deviation: 0.11
Number of observations: 30368

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 30368

Summary of vega:
Mean: 186.45
Median: 138.34
Standard deviation: 166.36
Number of observations: 30368

Summary of theta:
Mean: -34.3
Median: -25.01
Standard deviation: 42.8
Number of observations: 30368


In [41]:
features += ['strike', 'spot_close', 'divrate', 'vix', 'short_rate', 'delta', 'date'] #+ ['optid']
train_features = features[1:-1]

In [42]:
clear_df = df[features]

In [43]:
start_date = '2007-01'

y_prices = clear_df[clear_df['date'] > start_date]['midprice']

In [44]:
def get_splits(df, num=False):
    train_pc = 0.85

    df_len = df.shape[0]
    separator = int(np.floor(df_len*train_pc))
    rest = df_len - separator

    if num:
        return df[:separator], df[separator:]
    else:
        return np.append(np.ones(separator)*-1,np.zeros(rest))

## OLS Regression

In [15]:
ols_predictions = np.array([])
ols_r_2 = np.array([])
ols_r2_better = np.array([])

In [16]:
# OLS
for current_date in df[df['date'] > start_date]['date'].unique():
    
    x_train = clear_df[clear_df['date'] < current_date][train_features]
    y_train = clear_df[clear_df['date'] < current_date]['midprice']

    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']
    
    ols = LinearRegression()
    ols.fit(x_train, y_train)
    
    y_pred = ols.predict(x_test)
    
    # Write predictions to array
    ols_predictions = np.append(ols_predictions, y_pred)
    
    # Write R^2 to array
    ols_r_2 = np.append(ols_r_2, ols.score(x_train, y_train))
    
    ols_r2_better = np.append(ols_r2_better, r2_score(y_test, y_pred))

In [17]:
np.round(np.mean(ols_r_2), 3)

0.408

In [18]:
# MSE
np.round(mean_squared_error(y_prices, ols_predictions), 3)

19784.398

In [19]:
np.round(r2_score(y_prices, ols_predictions), 3)

0.329

In [20]:
np.round(np.mean(ols_r2_better), 3)

0.289

## Penalized Linear Regressions

### Ridge

In [21]:
ridge_predictions = np.array([])
ridge_r_2 = np.array([])
ridge_r2_better = np.array([])

In [22]:
# Make visualization

# ridge_cv.best_params_
# ridge_cv.best_score_


In [23]:
# Ridge
space = dict()

space['solver'] = ['auto','svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
space['alpha'] = np.array([10.0, 1.0, 0.1, 0.01, 0.001, 0])

for current_date in df[df['date'] > start_date]['date'].unique():
    temp = clear_df[clear_df['date'] < current_date]
    X, y = temp[train_features], temp['midprice']

    ps = PredefinedSplit(get_splits(temp))
    cv_scheme = list(ps.split())

    ridge = Ridge(scoring='r2', fit_intercept=True, normalize=True)

    ridge_cv = GridSearchCV(ridge, param_grid=space, scoring='r2', cv = cv_scheme)

    ridge_cv.fit(X, y)
    
    # train, validation = get_splits(clear_df[clear_df['date'] < '2007-02'], num = True)

    # x_train = train[train_features]
    # y_train = train['midprice']

    # x_valid = validation[train_features]
    # y_valid = validation['midprice']
    
    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']

    y_pred = ridge_cv.predict(x_test)

    # Write predictions to array
    ridge_predictions = np.append(ridge_predictions, y_pred)

    # Write R^2 to array
    ridge_r_2 = np.append(ridge_r_2, ridge_cv.score(X, y))
    
    ridge_r2_better = np.append(ridge_r2_better, r2_score(y_test, y_pred))

In [24]:
np.round(np.mean(ridge_r_2), 3)

0.388

In [25]:
np.round(mean_squared_error(y_prices, ridge_predictions), 3)

19767.199

In [26]:
np.round(r2_score(y_prices, ridge_predictions), 3)

0.329

In [27]:
np.round(np.mean(ridge_r2_better), 3)

0.307

### Lasso

In [28]:
lasso_predictions = np.array([])
lasso_r_2 = np.array([])
lasso_r2_better = np.array([])

In [29]:
# Lasso
space = dict()

space['alpha'] = np.array([10.0, 1.0, 0.1, 0.01, 0.001])

for current_date in df[df['date'] > start_date]['date'].unique():
    temp = clear_df[clear_df['date'] < current_date]
    X, y = temp[train_features], temp['midprice']

    ps = PredefinedSplit(get_splits(temp))
    cv_scheme = list(ps.split())

    lasso = Lasso(fit_intercept=True, normalize=True)
    
    lasso_cv = GridSearchCV(lasso, param_grid=space, scoring='r2', cv = cv_scheme)
    
    lasso_cv.fit(X, y)
        
    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']
    
    y_pred = lasso_cv.predict(x_test)
    
    # Write predictions to array
    lasso_predictions = np.append(lasso_predictions, y_pred)
    
    # Write R^2 to array
    lasso_r_2 = np.append(lasso_r_2, lasso_cv.score(X, y))
    
    lasso_r2_better = np.append(lasso_r2_better, r2_score(y_test, y_pred))

### ElasticNet

In [45]:
elastic_net_predictions = np.array([])
elastic_net_r_2 = np.array([])
elastic_net_r2_better = np.array([])

In [54]:
# ElasticNet
space = dict()

space['alpha'] = np.array([1.0, 0.01, 0.001, 0.0001])
space['l1_ratio'] = np.arange(0.95, 1, 0.005)

for current_date in df[df['date'] > start_date]['date'].unique():
    print(current_date)
    temp = clear_df[clear_df['date'] < current_date]
    X, y = temp[train_features], temp['midprice']

    ps = PredefinedSplit(get_splits(temp))
    cv_scheme = list(ps.split())
    
    elastic_net = ElasticNet(fit_intercept=True, normalize=True)
    
    elastic_net_cv = GridSearchCV(elastic_net, param_grid=space, scoring='r2', cv = cv_scheme)
    
    elastic_net_cv.fit(X, y)
    
    print(elastic_net_cv.best_params_)
    
    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']
    
    y_pred = elastic_net_cv.predict(x_test)
    
    # Write predictions to array
    elastic_net_predictions = np.append(elastic_net_predictions, y_pred)
    
    # Write R^2 to array
    elastic_net_r_2 = np.append(elastic_net_r_2, elastic_net_cv.score(X, y))
    
    elastic_net_r2_better = np.append(elastic_net_r2_better, r2_score(y_test, y_pred))

2007-02
{'alpha': 0.001, 'l1_ratio': 0.9650000000000001}
2007-03
{'alpha': 0.001, 'l1_ratio': 0.9650000000000001}
2007-04
{'alpha': 0.001, 'l1_ratio': 0.9650000000000001}
2007-05
{'alpha': 0.001, 'l1_ratio': 0.9650000000000001}
2007-06
{'alpha': 0.001, 'l1_ratio': 0.9700000000000001}
2007-07
{'alpha': 0.001, 'l1_ratio': 0.9700000000000001}
2007-08
{'alpha': 0.001, 'l1_ratio': 0.9700000000000001}
2007-09
{'alpha': 0.001, 'l1_ratio': 0.9750000000000001}
2007-10
{'alpha': 0.001, 'l1_ratio': 0.9750000000000001}
2007-11
{'alpha': 0.001, 'l1_ratio': 0.9800000000000001}
2007-12
{'alpha': 0.001, 'l1_ratio': 0.9800000000000001}
